<a href="https://colab.research.google.com/github/jimmy93029/Nanshan_tomb_image_segmentation.ipynb/blob/master/Predicting_mask_for_tifimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install package
( be aware that super-gradients sometimes change their package ralease version,
so please update super-gradients to the latest version to avoid some error about super-gradients)

In [1]:
%pip install segment-geospatial

In [ ]:
!pip install super-gradients==3.2.0

In [19]:
!pip install -q supervision

In [ ]:
!pip install aspose-words

In [3]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git

[Errno 2] No such file or directory: '{HOME}'
/content
/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


then restart the runtime ?

### inputs and setting

In [1]:
import torch

inputs = {
      "model_arch": 'yolo_nas_l',
      "num_classes": 2,
      "checkpoint_path": "/content/drive/MyDrive/project_NanShang/resources/average_model.pth",
      "sam_checkpoint_path": "/content/drive/MyDrive/project_NanShang/resources/sam_vit_h_4b8939.pth",
      "device": 'cuda' if torch.cuda.is_available() else "cpu",
      "confidence_threshold": 0.35,
      "sam_encoder_version": "vit_h",
      "source_tiffile": "/content/drive/MyDrive/project_NanShang/resources/NanShang_Tomb_cp.tif ",
      "tile_size": 700,
      "top_left_lat": 22.97494,
      "top_left_lon": 120.19544,
      "below_right_lat": 22.96717,
      "below_right_lon": 120.19775,

}

In [2]:
import os

STOREHOUSE = "storehouse"
SPLITED_TIFS_DIR = os.path.join(STOREHOUSE, "splited_tifs")
MASK_DIR = os.path.join(STOREHOUSE, "masks")

complete_mask_filename = "map_mask"
COMPLETE_MASK_JPG = os.path.join(STOREHOUSE, complete_mask_filename + ".jpg")
COMPLETE_MASK_TIFF = os.path.join(STOREHOUSE, complete_mask_filename + ".tiff")
COMPLETE_MASK_GEOTIFF = os.path.join(STOREHOUSE, "geo" + complete_mask_filename + ".tiff")

OUTPUT_DIR = "output"
data = {}

### initialize directory

In [6]:
import os

%cd {HOME}
os.makedirs(OUTPUT_DIR)
os.mkdir(STOREHOUSE)
os.mkdir(SPLITED_TIFS_DIR)
os.mkdir(MASK_DIR)

[Errno 2] No such file or directory: '{HOME}'
/content


### Cropping tif

In [7]:
from samgeo.common import split_raster

split_raster(inputs["source_tiffile"], SPLITED_TIFS_DIR, tile_size=inputs["tile_size"])

### predict bounding box

In [3]:
from super_gradients.training import models

def load_object_detection_(model_arch: str, num_classes: int, checkpoint_path: str, device: str):

    model = models.get(
        model_arch,
        num_classes=num_classes,
        checkpoint_path=checkpoint_path
    ).to(device)

    return model

[2023-08-16 13:32:33] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2023-08-16 13:32:35] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-08-16 13:32:40] INFO - utils.py - NumExpr defaulting to 4 threads.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-08-16 13:32:41] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-08-16 13:32:41] WARNING - export.py - Failed to import pytorch_quantization
[2023-08-16 13:32:41] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [ ]:
import tifffile
import numpy as np
import supervision as sv

tifs_dir = os.listdir(SPLITED_TIFS_DIR)
box_predictor = load_object_detection_(inputs["model_arch"], inputs["num_classes"],
                         inputs["checkpoint_path"], inputs["device"])
data["detections"] = {}

num_tombs = 0
for tif in tifs_dir:
    # read tif as numpy array
    image = tifffile.imread(os.path.join(SPLITED_TIFS_DIR, tif))
    img_array = np.array(image)

    # predict bounding box
    result = list(box_predictor.predict(img_array, conf=inputs["confidence_threshold"]))[0]
    boxes = result.prediction.bboxes_xyxy
    num_tombs += len(boxes)  # compute the number of tombs

    # make sv.Detections
    detection = sv.Detections(
             xyxy=boxes,
             confidence=result.prediction.confidence,
             class_id=result.prediction.labels.astype(int)
          )
    data["detections"][tif] = detection

    # finally we get the total number of tombs
    print(f"the number of tombs is {num_tombs}")
    data["num_tombs"] = num_tombs




### predict_mask

In [27]:
from segment_anything import sam_model_registry, SamPredictor

def segment(sam_predictor: "SamPredictor", image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def load_segment_anything_model(sam_encoder_version, sam_checkpoint_path, device):
    sam = sam_model_registry[sam_encoder_version](checkpoint=sam_checkpoint_path).to(device=device)
    sam_predictor = SamPredictor(sam)
    return sam_predictor

In [28]:
from PIL import Image

tifs_dir = os.listdir(SPLITED_TIFS_DIR)
sam_predictor = load_segment_anything_model(inputs["sam_encoder_version"], inputs["sam_checkpoint_path"],
                                    inputs["device"])
mask_annotator = sv.MaskAnnotator()

for tif in tifs_dir:
    # read tif as numpy array
    image = tifffile.imread(os.path.join(SPLITED_TIFS_DIR, tif))
    img_array = np.array(image)

    # make detection's mask
    data["detections"][tif].mask = segment(
          sam_predictor=sam_predictor,
          image=img_array.copy(),
          xyxy=data["detections"][tif].xyxy
          )

    # produce mask and save as image
    blank = np.zeros_like(img_array)
    mask = mask_annotator.annotate(scene=blank, detections=data["detections"][tif])
    img = Image.fromarray(mask, "RGB")
    img.save(os.path.join(MASK_DIR, "mask" + tif[4:-4] + ".jpg"))

### merge tif

In [29]:
from osgeo import gdal

def read_tif(tif_path):
    ds = gdal.Open(tif_path)
    row = ds.RasterXSize
    col = ds.RasterYSize
    return row, col

In [31]:
width, height = read_tif(inputs["source_tiffile"])
size = inputs["tile_size"]
img = Image.new("RGB", (width, height))
mask_dir = os.listdir(MASK_DIR)

# collage small image to a complete image
for mask_filename in mask_dir:
    im = Image.open(os.path.join(MASK_DIR, mask_filename))
    name_list = mask_filename[:-4].split("_")  # ex. turn mask_0_11.jpg into ['mask', '0', '11']
    i = int(name_list[1])      # the 1th value in name_list indicate the order in x ray
    j = int(name_list[2])      # the 2th value in name_list indicate the order in y ray (see samgeo.common.split_raster())
    img.paste(im, (i * size, j * size))

img.save(COMPLETE_MASK_JPG)


### image to geotiff

In [3]:
import aspose.words as aw
import rasterio
from rasterio.transform import from_origin

def jpg2tiff():
    doc = aw.Document()
    builder = aw.DocumentBuilder(doc)
    shape = builder.insert_image(COMPLETE_MASK_JPG)
    shape.image_data.save(COMPLETE_MASK_TIFF)

def tiff2goetiff(below_right_lat, top_left_lat, below_right_lon, top_left_lon):
    # Read the input TIFF
    with rasterio.open(COMPLETE_MASK_GEOTIFF) as src:
        data = src.read()
        dtype = src.dtypes[0]
        count = src.count
        height, width = src.height, src.width

    # compute horizontal pixel size and vertical pixel size in degrees
    dis_lat = below_right_lat - top_left_lat
    dis_lon = below_right_lon - top_left_lon
    pixel_size_x = dis_lon / width  # Example: horizontal pixel size in degrees
    pixel_size_y = dis_lat / height  # Example: vertical pixel size in degrees

    # Define geospatial information
    crs = 'EPSG:4326'  # Example: WGS 84

    # Define the geotransform
    transform = from_origin(top_left_lon, top_left_lat, pixel_size_x, pixel_size_y)

    # Write the GeoTIFF file
    with rasterio.open(
            COMPLETE_MASK_GEOTIFF,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=count,
            dtype=dtype,
            crs=crs,
            transform=transform,
    ) as dst:
        dst.write(data)

In [ ]:
jpg2tiff()
tiff2goetiff(inputs["below_right_lat"], inputs["top_left_lat"],
                  inputs["below_right_lon"], inputs["top_left_lon"])

### geotiff to shp

In [35]:
from samgeo.common import raster_to_shp

raster_to_shp(COMPLETE_MASK_GEOTIFF, OUTPUT_DIR)

[2023-08-16 15:03:48] INFO - __init__.py - GDAL signalled an error: err_no=4, msg='storehouse/geomap_mask.tiff: No such file or directory'


RasterioIOError: ignored